In [1]:
import os
import glob
import datetime
import numpy as np
import pickle
import pandas as pd

startDate = '20201216'
endDate = '20201218'

readPath = r'/mnt/orderLog/data/***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
print(dataPathLs)

rawOrderLog = []
for thisPath in dataPathLs:
    data = pickle.load(open(thisPath, 'rb'))
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

# startDate = '20201216'
# endDate = '20201218'

# readPath = '/mnt/equityTradeLogs'
# dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
# dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
# dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

# rawOrderLog = []
# for thisPath in dataPathLs:
#     data = pd.read_csv(thisPath)
#     data = data.rename(columns={'mdClockAtArrival': 'caamd'})
#     rawOrderLog += [data]
# rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
df = orderLog.copy()


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['mrsb90'] == '-'])
orderLog = orderLog[orderLog['mrsb90'] != '-']
orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['aaa'] == '-'])
orderLog = orderLog[orderLog['aaa'] != '-']
orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

['/mnt/orderLog/data/20201216.pkl' '/mnt/orderLog/data/20201217.pkl'
 '/mnt/orderLog/data/20201218.pkl']


'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
270764,20201216,zs_64_01,6480,2000677,-1,1,49418.0,0,0,-1.0,1.001710e+05
323257,20201216,zs_54_01,5474,2002094,-1,7,36274.0,0,0,-1.0,-1.000000e+00
368535,20201216,zs_52_09,5291,2002375,-1,1,53790.0,0,0,-1.0,1.302009e+06
392174,20201216,zs_52_09,5291,2002506,-1,1,34313.0,0,0,-1.0,5.782000e+04
432420,20201216,zs_64_01,6480,2002729,-1,1,47305.0,0,0,-1.0,8.880200e+04
482657,20201216,zs_64_01,6480,2002956,-1,1,49543.0,0,0,-1.0,1.007390e+05
522525,20201216,zs_64_01,6480,2300151,-1,1,47491.0,0,0,-1.0,8.986500e+04
546021,20201216,zs_66_01,6634,2300272,-1,1,52636.0,0,0,-1.0,1.812810e+10
570919,20201216,zs_64_01,6480,2300374,-1,1,51532.0,0,0,-1.0,1.227210e+05
581196,20201216,zs_52_06,5269,2300410,-1,1,35172.0,0,0,-1.0,2.519470e+05


There are orders with all things same except sdd
        ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
403912   3686264.0   0.00044                    0           100   
404193  18252604.0  0.001406                    0           100   
435894   3688220.0  0.000419                    0           100   
435917  14404537.0  0.000657                    0           100   

        absOrderSizeCumFilled  accCode  ars  caamd  cancellationPending  \
403912                      0   966301  0.0      0                  0.0   
404193                      0   966301  0.0      0                  0.0   
435894                      0   966301  0.0      0                  0.0   
435917                      0   966301  0.0      0                  0.0   

              cfe  ...  TradePriceLs  TradeQtyLs   beta_60  adjMid_F30s  \
403912  506777.82  ...           NaN         NaN  1.224385    26.348345   
404193  147834.78  ...           NaN         NaN  1.224385    26.348345   
435894  568630

/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,...,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status


11. Orders with size > 80w or notional > 800w


/home/work516/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,date,secid,colo,accCode,updateType,vai,absOrderSize,orderDirection,mrstaat,mrstauc,sdd,ApplSeqNum,sta
1078,20201216,1600021,zt_96_01,966301,0,1221426.0,200,1,0.0,0.0,111922000.0,0.0,else
1079,20201216,1600021,zt_96_01,966301,2,1221426.0,200,1,0.0,0.0,-1.0,0.0,else
1080,20201216,1600021,zt_96_01,966301,4,1221426.0,200,1,0.0,0.0,-1.0,0.0,else
1081,20201216,1600021,zt_96_01,966301,0,1223026.0,200,1,0.0,0.0,111922000.0,1910877.0,else
1082,20201216,1600021,zt_96_01,966301,2,1223026.0,200,1,0.0,0.0,-1.0,0.0,else
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2161383,20201218,2300376,zt_96_01,966301,2,14692744.0,400,-1,0.0,0.0,-1.0,0.0,else
2161384,20201218,2300376,zt_96_01,966301,4,14692744.0,400,-1,0.0,0.0,-1.0,0.0,else
2190530,20201218,2300459,zt_96_01,966301,0,7120000.0,100,-1,0.0,0.0,95304830.0,4916259.0,else
2190531,20201218,2300459,zt_96_01,966301,2,7120000.0,100,-1,0.0,0.0,-1.0,0.0,else


mrstaat  mrstauc
1000.0   0.0         81408
3000.0   1000.0     329468
         2000.0      21151
         3000.0       7026
11000.0  10000.0      5086
         20000.0       319
         30000.0       226
13000.0  11000.0    117571
         12000.0      3549
         13000.0       763
         21000.0      2736
         22000.0      3078
         23000.0       591
         31000.0      1276
         32000.0      1102
         33000.0      1235
Name: date, dtype: int64

In [12]:
import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def read_beta_daily(db, name, start_date=None, end_date=None, skey=None, interval=None, col=None, return_sdi=True): 
    collection = db[name] 
    # Build projection 
    prj = {'_id': 0} 
    if col is not None: 
        if return_sdi: 
            col = ['skey', 'date'] + col 
        for col_name in col: 
            prj[col_name] = 1 
 
    # Build query 
    query = {} 
    if skey is not None: 
        query['skey'] = {'$in': skey} 
    if interval is not None: 
        query['interval'] = {'$in': interval} 
    if start_date is not None: 
        if end_date is not None: 
            query['date'] = {'$gte': start_date, '$lte': end_date} 
        else: 
            query['date'] = {'$gte': start_date} 
    elif end_date is not None: 
        query['date'] = {'$lte': end_date} 
 
    # Load data 
    cur = collection.find(query, prj) 
    df = pd.DataFrame.from_records(cur) 
    if df.empty: 
        df = pd.DataFrame() 
    else: 
        df = df.sort_values(by=['date','skey']) 
    return df  

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)
tag = read_memb_daily(db1, 'index_memb', 20201124, 20201217, index_id=[1000300, 1000852, 1000905, 1000985])

In [17]:
tag = tag.rename(columns={'skey':'secid'})
orderLog = pd.merge(orderLog, tag[['date', 'secid', 'index_name']], on=['date', 'secid'], how='left')

In [19]:
orderLog[(orderLog['exchange'] == 'SZE') & (orderLog['accCode'].astype(str).str[:2] == '88')].groupby(['colo', 'accCode'])['index_name'].unique()

colo      accCode
zs_88_04  8856            [IC]
          8865       [CSI1000]
zt_88_03  8833       [CSI1000]
zt_88_06  8854       [CSI1000]
Name: index_name, dtype: object

In [20]:
checkLog = orderLog[(orderLog["updateType"] == 0)]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
# SSE = checkLog[checkLog['secid'] < 2000000]
# SSE["exchange"] = "SH"

# c1 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
# c2 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
# c3 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].count().reset_index()
# c4 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["date"].unique().str.len().reset_index()
# c5 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].std().reset_index()

# re1 = pd.merge(c3, c1, on=['colo', 'exchange', 'accCode', 'mrstaat'])
# re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
# re1 = pd.merge(re1, c2, on=['colo', 'exchange', 'accCode', 'mrstaat'])
# re1 = re1.rename(columns = {'internal_latency': 'median'})
# re1 = pd.merge(re1, c4, on=['colo', 'exchange', 'accCode', 'mrstaat'])
# re1 = pd.merge(re1, c5, on=['colo', 'exchange', 'accCode', 'mrstaat'])
# re1 = re1.rename(columns = {'internal_latency': 'std'})


# # re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# # re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
# re = pd.concat([re1]).reset_index(drop=True)


# for col in ['median', '95 percentile']:
#     re[col] = re[col].astype(int)
# for col in ['std']:
#     re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
# from IPython.display import display, HTML

# display(HTML(re[re['colo'].str[:5] == 'zt_52'].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))
# display(HTML(re[re['colo'].str[:5] == 'zs_52'].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))

SZE = checkLog[checkLog['secid'] >= 2000000]
SZE["exchange"] = "SZ"

c1 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
c2 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
c3 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].count().reset_index()
c4 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["date"].unique().str.len().reset_index()
c5 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = pd.merge(re1, c5, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re[(re['accCode'] == 8856) | (re['accCode'] // 100 == 88)].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))


/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [53]:
checkLog = orderLog[(orderLog["updateType"] == 0)]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
# SSE = checkLog[checkLog['secid'] < 2000000]
# SSE["exchange"] = "SH"

# c1 = SSE.groupby(['colo', 'exchange', 'accCode', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode'])["internal_latency"].mean().reset_index()
# c2 = SSE.groupby(['colo', 'exchange', 'accCode', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange', 'accCode'])["internal_latency"].mean().reset_index()
# c3 = SSE.groupby(['colo', 'exchange', 'accCode'])["internal_latency"].count().reset_index()
# c4 = SSE.groupby(['colo', 'exchange', 'accCode'])["date"].unique().str.len().reset_index()
# c5 = SSE.groupby(['colo', 'exchange', 'accCode', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode'])["internal_latency"].std().reset_index()

# re1 = pd.merge(c3, c1, on=['colo', 'exchange', 'accCode'])
# re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
# re1 = pd.merge(re1, c2, on=['colo', 'exchange', 'accCode'])
# re1 = re1.rename(columns = {'internal_latency': 'median'})
# re1 = pd.merge(re1, c4, on=['colo', 'exchange', 'accCode'])
# re1 = pd.merge(re1, c5, on=['colo', 'exchange', 'accCode'])
# re1 = re1.rename(columns = {'internal_latency': 'std'})


# # re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# # re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
# re = pd.concat([re1]).reset_index(drop=True)


# for col in ['median', '95 percentile']:
#     re[col] = re[col].astype(int)
# for col in ['std']:
#     re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
# from IPython.display import display, HTML

# display(HTML(re[re['colo'].str[:5] == 'zt_52'].groupby(['colo', 'exchange', 'accCode'])["count", "median", "95 percentile", "std"].first().to_html()))
# display(HTML(re[re['colo'].str[:5] == 'zs_52'].groupby(['colo', 'exchange', 'accCode'])["count", "median", "95 percentile", "std"].first().to_html()))

SZE = checkLog[checkLog['secid'] >= 2000000]
SZE["exchange"] = "SZ"

c1 = SZE.groupby(['colo', 'exchange', 'accCode', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode'])["internal_latency"].mean().reset_index()
c2 = SZE.groupby(['colo', 'exchange', 'accCode', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange', 'accCode'])["internal_latency"].mean().reset_index()
c3 = SZE.groupby(['colo', 'exchange', 'accCode'])["internal_latency"].count().reset_index()
c4 = SZE.groupby(['colo', 'exchange', 'accCode'])["date"].unique().str.len().reset_index()
c5 = SZE.groupby(['colo', 'exchange', 'accCode', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['colo', 'exchange', 'accCode'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['colo', 'exchange', 'accCode'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['colo', 'exchange', 'accCode'])
re1 = pd.merge(re1, c5, on=['colo', 'exchange', 'accCode'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re[(re['accCode'] == 8856) | (re['accCode'] // 100 == 88)].groupby(['colo', 'exchange', 'accCode'])["count", "median", "95 percentile", "std"].first().to_html()))


/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [59]:
checkLog = orderLog[(orderLog["updateType"] == 0)]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
# SSE = checkLog[checkLog['secid'] < 2000000]
# SSE["exchange"] = "SH"

# c1 = SSE.groupby(['colo_broker', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo_broker', 'exchange'])["internal_latency"].mean().reset_index()
# c2 = SSE.groupby(['colo_broker', 'exchange', "date"])["internal_latency"].median().reset_index().groupby(['colo_broker', 'exchange'])["internal_latency"].mean().reset_index()
# c3 = SSE.groupby(['colo_broker', 'exchange'])["internal_latency"].count().reset_index()
# c4 = SSE.groupby(['colo_broker', 'exchange'])["date"].unique().str.len().reset_index()
# c5 = SSE.groupby(['colo_broker', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo_broker', 'exchange'])["internal_latency"].std().reset_index()

# re1 = pd.merge(c3, c1, on=['colo_broker', 'exchange'])
# re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
# re1 = pd.merge(re1, c2, on=['colo_broker', 'exchange'])
# re1 = re1.rename(columns = {'internal_latency': 'median'})
# re1 = pd.merge(re1, c4, on=['colo_broker', 'exchange'])
# re1 = pd.merge(re1, c5, on=['colo_broker', 'exchange'])
# re1 = re1.rename(columns = {'internal_latency': 'std'})


# # re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_broker_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# # re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_broker_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
# re = pd.concat([re1]).reset_index(drop=True)


# for col in ['median', '95 percentile']:
#     re[col] = re[col].astype(int)
# for col in ['std']:
#     re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
# from IPython.display import display, HTML

# display(HTML(re[re['colo_broker'].str[:5] == 'zt_52'].groupby(['colo_broker', 'exchange'])["count", "median", "95 percentile", "std"].first().to_html()))
# display(HTML(re[re['colo_broker'].str[:5] == 'zs_52'].groupby(['colo_broker', 'exchange'])["count", "median", "95 percentile", "std"].first().to_html()))

SZE = checkLog[checkLog['secid'] >= 2000000]
SZE["exchange"] = "SZ"

c1 = SZE.groupby(['colo_broker', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo_broker', 'exchange'])["internal_latency"].mean().reset_index()
c2 = SZE.groupby(['colo_broker', 'exchange', "date"])["internal_latency"].median().reset_index().groupby(['colo_broker', 'exchange'])["internal_latency"].mean().reset_index()
c3 = SZE.groupby(['colo_broker', 'exchange'])["internal_latency"].count().reset_index()
c4 = SZE.groupby(['colo_broker', 'exchange'])["date"].unique().str.len().reset_index()
c5 = SZE.groupby(['colo_broker', 'exchange', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo_broker', 'exchange'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['colo_broker', 'exchange'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['colo_broker', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['colo_broker', 'exchange'])
re1 = pd.merge(re1, c5, on=['colo_broker', 'exchange'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_broker_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_broker_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re.groupby(['colo_broker', 'exchange'])["count", "median", "95 percentile", "std"].first().to_html()))


/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,,count,median,95 percentile,std
colo_broker,exchange,,,,
zs_52,SZ,166048,57,122,5.57
zs_53,SZ,19716,64,129,1.53
zs_54,SZ,17281,29,43,3.21
zs_62,SZ,4901,68,172,42.78
zs_64,SZ,8976,27,49,4.10
zs_66,SZ,46380,63,113,11.85
zs_72,SZ,440,54,81,49.85
zs_88,SZ,7385,54,88,6.03
zs_89,SZ,18400,69,119,4.59


In [54]:
from IPython.display import display, HTML

display(HTML(re.groupby(['colo', 'exchange', 'accCode'])["count", "median", "95 percentile", "std"].first().to_html()))


/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
checkLog = orderLog[(orderLog['updateType'] == 0) & (~orderLog['caa_orderLog'].isnull())].drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)

In [30]:
cc1 = checkLog[checkLog['colo'].isin(['zs_88_04'])]
cc1 = cc1.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
cc1 = cc1.reset_index(drop=True)
cc1['ordering'] = cc1.index
cc1['time_diff'] = cc1['caa_orderLog'] - cc1['start_time']
cc1['colo1'] = cc1['colo'].str[:2] + cc1['colo'].str[3:5] + cc1['colo'].str[6:8]
cc1['colo_broker'] = cc1['colo1'] + '_' + cc1["accCode"].astype(int).astype(str)
cc1['accCode'] = cc1['accCode'].fillna(0).astype(int)
cc1['sta'] = np.where(cc1['mrstaat'] == 1000, 'staone', np.where(
cc1['mrstaat'] == 3000, 'statwo', np.where(
cc1['mrstaat'] == 11000, 'stathree13', 'stathree23')))

checkLog1 = cc1[cc1['sta'] == 'statwo']
checkLog2 = cc1[(cc1['sta'] == 'staone')]

from IPython.display import display, HTML
re1 = checkLog1.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog1.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['mrstaat', 'colo_broker', 'colo', '10%']], on=['mrstaat', 'colo_broker', 'colo'])
c1 = checkLog1.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['mrstaat', 'colo_broker', 'colo', '90%']], on=['mrstaat', 'colo_broker', 'colo'])

display(HTML(re1[['mrstaat', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['mrstaat', 'colo_broker']).first().to_html()))

from IPython.display import display, HTML
re1 = checkLog2.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog2.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['mrstaat', 'colo_broker', 'colo', '10%']], on=['mrstaat', 'colo_broker', 'colo'])
c1 = checkLog2.groupby(['mrstaat', 'colo_broker', 'colo'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['mrstaat', 'colo_broker', 'colo', '90%']], on=['mrstaat', 'colo_broker', 'colo'])

display(HTML(re1[['mrstaat', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['mrstaat', 'colo_broker']).first().to_html()))

In [10]:
checkLog = orderLog[(orderLog['updateType'] == 0) & (~orderLog['caa_orderLog'].isnull())].drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog.reset_index(drop=True)
checkLog['ordering'] = checkLog.index
checkLog['time_diff'] = checkLog['caa_orderLog'] - checkLog['start_time']
checkLog['colo1'] = checkLog['colo'].str[:2]
checkLog['colo_broker'] = checkLog['colo1'] + '_' + checkLog["accCode"].astype(str).str[:2]
ol = pd.merge(checkLog[checkLog['sta'] == '1. staone'], checkLog[checkLog['sta'] == '2. statwo'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
ol1 = pd.merge(checkLog[checkLog['sta'] == '1. staone'], checkLog[checkLog['sta'] == '3. sta300'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
ol2 = pd.merge(checkLog[checkLog['sta'] == '2. statwo'], checkLog[checkLog['sta'] == '3. sta300'][['colo', 'accCode']].drop_duplicates(),
         on=['colo', 'accCode'], how='inner')['ordering'].unique()
checkLog1 = checkLog[(checkLog['sta'] == '1. staone') & (~checkLog['ordering'].isin(ol)) & (~checkLog['ordering'].isin(ol1))]
checkLog2 = checkLog[(checkLog['sta'] == '2. statwo') & (~checkLog['ordering'].isin(ol2))]
checkLog3 = checkLog[checkLog['sta'] == '3. sta300']

from IPython.display import display, HTML
re1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog1.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))

from IPython.display import display, HTML
re1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog2.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))

from IPython.display import display, HTML
re1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
c1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"10%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '10%']], on=['sta', 'colo', 'colo_broker'])
c1 = checkLog3.groupby(['sta', 'colo', 'colo_broker'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
c1 = c1.rename(columns={"time_diff":"90%"})
re1 = pd.merge(re1, c1[['sta', 'colo', 'colo_broker', '90%']], on=['sta', 'colo', 'colo_broker'])

display(HTML(re1[['sta', 'colo', 'colo_broker', 'count',  '10%', '25%', '50%', '75%', '90%']].groupby(['sta', 'colo', 'colo_broker']).first().sort_values(by='50%').to_html()))


In [6]:
orderLog = orderLog[~orderLog['mrstauc'].isnull()]
orderLog['sta_uc'] = orderLog['sta'] + '_' + orderLog['mrstauc'].astype(int).astype(str)
orderLog["server"] = orderLog["colo"].apply(lambda x: x.split("_")[0] + x.split("_")[1] + x.split("_")[2])
orderLog["server_account"] = orderLog["server"] + '_' + orderLog['accCode'].astype('str')
checkLog = orderLog[orderLog['updateType'].isin([0, 3, 4])].reset_index(drop=True)
checkLog = checkLog[checkLog['date'] >= 20201216]
checkLog = checkLog[checkLog['duration'] <= 1e6]

checkLog["time"] = checkLog["clock"].apply(lambda x: x.strftime("%H%M%S")).astype(int)
# ol = checkLog[(checkLog["updateType"] == 0) & (~checkLog["ars"].isin([121, 221, 321, 131, 231, 331])) & (checkLog["time"]>= 93000)
#              & (checkLog["time"] < 93300)]["order"].unique()

sta_list = orderLog['sta_uc'].unique()

result = {}
for col in ['date', 'broker1', 'broker2', 'isFast']:
    result[col] = []
            
for i in sta_list:
    checkLog1 = checkLog[checkLog['sta_uc'] == i]
    checkLog1['accountNum'] = checkLog1.groupby(['date', 'secid', 'vai'])['server_account'].transform('nunique')
    checkLog1 = checkLog1[checkLog1['accountNum'] >= 2]
    checkLog1['cumFillSize'] = checkLog1.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
    checkLog1['status'] = np.where(checkLog1['cumFillSize'] == 0, 2, 
                                    np.where(checkLog1['cumFillSize'] < checkLog1['absOrderSize'], 1, 0))
    checkLog1 = checkLog1.groupby(['group', 'order'])[['date', 'server_account', 'secid', 'vai', 'status']].first().reset_index()

    for exchg in ['SZE']:

        print(exchg)

        if exchg == 'SSE':
            exchangeLog = checkLog1[checkLog1['secid'] < 2000000]
        else:
            exchangeLog = checkLog1[checkLog1['secid'] >= 2000000]

        for group, groupData in exchangeLog.groupby(['group']):
            date = groupData['date'].values[0]
            brokerLs = groupData['server_account'].values
            if len(brokerLs) == 0:
                continue
            statusLs = groupData['status'].values
            ixLs = [i for i in range(len(brokerLs))]
            for k, broker1, status1 in zip(ixLs[:-1], brokerLs[:-1], statusLs[:-1]):
                for broker2, status2 in zip(brokerLs[k+1:], statusLs[k+1:]):
                    if broker1 != broker2:
                        result['date'] += [date, date]
                        result['broker1'] += [broker1, broker2]
                        result['broker2'] += [broker2, broker1]

                        if status1 < status2:
                            result['isFast'] += [1, 0]
                        elif status1 > status2:
                            result['isFast'] += [0, 1]
                        else:
                            result['isFast'] += [-1, -1]                

result = pd.DataFrame(result)
result = result[result['isFast'] != -1]
result['count'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('count')
result['fasterSum'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
result['faster'] = result['fasterSum']/result['count']


summary = pd.DataFrame()
summaryCols = []
countCols = []
fasterCols = []
for date in result['date'].unique():
    a = result[result['date'] == date].reset_index(drop=True)
    a['count'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('count')
    a['fasterSum'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
    a['faster'] = a['fasterSum']/a['count']
    a = a.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
    a.columns = ['broker1', 'broker2', '%s count'%date, '%s faster'%date]
    summaryCols += ['%s count'%date, '%s faster'%date]
    countCols += ['%s count'%date]
    fasterCols += ['%s faster'%date]

    if summary.empty:
        summary = a.copy()
    else:
        summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')

a = result.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
a.columns = ['broker1', 'broker2', 'total count', 'total faster']
summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')
summaryCols = ['total count', 'total faster'] + summaryCols

countCols += ['total count']
fasterCols += ['total faster']
summary['b1'] = summary['broker1'].str[7:11].astype(int) // 100
summary['b2'] = summary['broker2'].str[7:11].astype(int) // 100
#     summary = summary.groupby(['broker1', 'broker2'])[summaryCols].first()
for col in countCols:
    summary[col] = summary[col].fillna(0)
    summary[col] = summary[col].astype('int64')

for col in fasterCols:
    summary[col] = summary[col].fillna(-1)
    summary[col] = summary[col].apply(lambda x: '%.0f%%'%(x*100))
    summary[col] = np.where(summary[col] == '-100%', ' ', summary[col])

#     summary = summary.rename(columns={"broker1":"account1", "broker2":"account2"})
#     summary = summary.groupby(['account1', 'account2'])[summaryCols].first()
#     from IPython.display import display, HTML
#     display(HTML(summary.iloc[:, :2].to_html()))
#     summary["tt"]=summary[["broker1", "broker2"]].min(axis=1).astype(str) + ' - ' + summary[["broker1", "broker2"]].max(axis=1).astype(str)
#     summary = summary.sort_values(by=["tt", "broker1"])
#     s1 = summary.groupby("tt").first().reset_index(drop=True)
s1 = summary
display(s1[(s1["broker1"] == 'zs7201_7293')])

/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE


,broker1,broker2,20201216 count,20201216 faster,20201217 count,20201217 faster,20201218 count,20201218 faster,total count,total faster,b1,b2
632,zs7201_7293,zs5206_5332,0,,12,83%,9,11%,21,52%,72,53
633,zs7201_7293,zs5206_5384,0,,3,0%,0,,3,0%,72,53
634,zs7201_7293,zs5208_5229,0,,18,0%,8,0%,26,0%,72,52
635,zs7201_7293,zs5208_5276,0,,34,0%,33,0%,67,0%,72,52
636,zs7201_7293,zs5208_5292,0,,6,0%,6,0%,12,0%,72,52
637,zs7201_7293,zs5208_5377,0,,3,33%,1,0%,4,25%,72,53
638,zs7201_7293,zs5208_5386,0,,7,43%,5,20%,12,33%,72,53
639,zs7201_7293,zs5209_5289,0,,11,0%,13,0%,24,0%,72,52
640,zs7201_7293,zs5209_5291,0,,13,0%,21,0%,34,0%,72,52
641,zs7201_7293,zs8804_8856,0,,7,0%,12,0%,19,0%,72,88


In [13]:
a = orderLog[(orderLog['updateType'] == 0) & (orderLog['exchange'] == 'SZE')].groupby('colo_broker')['orderNtl'].sum().reset_index()
b = orderLog[(orderLog['updateType'] == 4) & (orderLog['exchange'] == 'SZE')].groupby('colo_broker')['tradeNtl'].sum().reset_index()
re = pd.merge(a, b, on='colo_broker')
re['fill rate'] = re['tradeNtl'] / re['orderNtl']
re['fill rate'] = re['fill rate'].apply(lambda x: '%.1f%%'%(100*x))
re

,colo_broker,orderNtl,tradeNtl,fill rate
0,zs_52,7.745017e+09,5.873706e+09,75.8%
1,zs_53,1.602981e+09,1.134562e+09,70.8%
2,zs_54,3.530350e+08,2.702628e+08,76.6%
3,zs_62,3.052436e+08,1.134728e+08,37.2%
4,zs_64,1.825632e+08,1.114928e+08,61.1%
5,zs_66,1.625177e+09,1.336567e+09,82.2%
6,zs_72,2.200249e+06,1.272235e+06,57.8%
7,zs_88,1.588699e+08,1.108767e+08,69.8%
8,zs_89,1.498838e+09,9.668653e+08,64.5%
9,zs_92,2.311351e+07,1.644593e+07,71.2%


In [7]:
s1[(s1["broker1"] == 'zs7201_7293')][['broker1', 'broker2', '20201217 count', '20201217 faster', '20201218 count',
       '20201218 faster', 'total count', 'total faster']].sort_values(by='total count', ascending=False)

,broker1,broker2,20201217 count,20201217 faster,20201218 count,20201218 faster,total count,total faster
635,zs7201_7293,zs5208_5276,34,0%,33,0%,67,0%
640,zs7201_7293,zs5209_5291,13,0%,21,0%,34,0%
634,zs7201_7293,zs5208_5229,18,0%,8,0%,26,0%
639,zs7201_7293,zs5209_5289,11,0%,13,0%,24,0%
632,zs7201_7293,zs5206_5332,12,83%,9,11%,21,52%
641,zs7201_7293,zs8804_8856,7,0%,12,0%,19,0%
636,zs7201_7293,zs5208_5292,6,0%,6,0%,12,0%
638,zs7201_7293,zs5208_5386,7,43%,5,20%,12,33%
642,zs7201_7293,zs9608_9754,9,0%,3,0%,12,0%
637,zs7201_7293,zs5208_5377,3,33%,1,0%,4,25%


In [ ]:
s1[(s1["broker1"] == 'zs_72')][['broker1', 'broker2', '20201217 count', '20201217 faster', '20201218 count',
       '20201218 faster', 'total count', 'total faster']].sort_values(by='total count', ascending=False)

In [51]:
rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] == 7293)].groupby(['date', 'mrstaat'])['orderPrice'].size()

date      mrstaat
20201216  3000.0      38
20201217  1000.0      36
          3000.0     150
20201218  1000.0     115
          3000.0     101
Name: orderPrice, dtype: int64

In [52]:
orderLog[(orderLog['accCode'] == 7293) & (orderLog['updateType'] == 0)].groupby(['date', 'sta'])['orderPrice'].size()

date      sta      
20201216  2. statwo     38
20201217  1. staone     36
          2. statwo    150
20201218  1. staone    115
          2. statwo    101
Name: orderPrice, dtype: int64